# EasyUQ on weather example

    1. Single gaussian applied to WeatherBench data (here: CNN)
    2. Smooth EasyUQ applied to WeatherBench data (here: CNN)
    3. Conformal prediction applied to WeatherBench data (here: CNN)

In [1]:
import numpy as np
import pandas as pd
import xarray as xr
import isodisreg 
from isodisreg import idr
from scipy import stats
import properscoring as ps
from helper_functions import single_gaussian_optim, log_norm

### 1. Single gaussian

Get CNN forecast from WeatherBench to apply single gaussian and smooth EasyUQ

In [6]:
# load via xarray:
cnn_forecast_train = xr.open_dataset('./data/cnn_fct_train.nc')
cnn_forecast_test = xr.open_dataset('./data/cnn_fct_test.nc')
y_train = xr.open_dataset('./data/cnn_obs_train.nc')
y_test = xr.open_dataset('./data/cnn_obs_test.nc')

# select one grid point:
lat = 8.4375
lon = 180
cnn_fct_grid_train = cnn_forecast_train.sel(lat = lat, lon = lon)['t'].values
cnn_fct_grid_test = cnn_forecast_test.sel(lat = lat, lon = lon)['t'].values
y_grid_train = y_train.sel(lat = lat, lon = lon)['t'].values
y_grid_test = y_test.sel(lat = lat, lon = lon)['t'].values

Run single gaussian method

In [7]:
sigma = single_gaussian_optim(y_grid_train, cnn_fct_grid_train)
crpss = np.mean(ps.crps_gaussian(y_grid_test, mu=cnn_fct_grid_test, sig=sigma)) 
ll = log_norm(y_grid_test, cnn_fct_grid_test, sigma)
print('Fitted sigma = %f' %sigma)
print('CRPS of single gaussian: %f' %crpss)
print('log score: %f' %ll)

Fitted sigma = 0.827519
CRPS of single gaussian: 0.401656
log score: 1.114682


### 2. Smooth EasyUQ via One-fit approach

Get CNN forecast from WeatherBench to apply single gaussian and smooth EasyUQ

In [2]:
from helper_functions import optimize_paras_onefit, llscore_idr

In [3]:
# load via xarray:
cnn_forecast_train = xr.open_dataset('./data/cnn_fct_train.nc')
cnn_forecast_test = xr.open_dataset('./data/cnn_fct_test.nc')
y_train = xr.open_dataset('./data/cnn_obs_train.nc')
y_test = xr.open_dataset('./data/cnn_obs_test.nc')

# select one grid point:
lat = 8.4375
lon = 180
cnn_fct_grid_train = cnn_forecast_train.sel(lat = lat, lon = lon)['t'].values
cnn_fct_grid_test = cnn_forecast_test.sel(lat = lat, lon = lon)['t'].values
y_grid_train = y_train.sel(lat = lat, lon = lon)['t'].values
y_grid_test = y_test.sel(lat = lat, lon = lon)['t'].values

In [4]:
# Fit EasyUQ
fitted_idr = idr(y_grid_train, pd.DataFrame({"fore": cnn_fct_grid_train}, columns=["fore"]))
preds_train = fitted_idr.predict(pd.DataFrame({"fore": cnn_fct_grid_train}, columns=["fore"]))
preds_test = fitted_idr.predict(pd.DataFrame({"fore": cnn_fct_grid_test}, columns=["fore"]))

In [ ]:
crps_idr = np.mean(preds_test.crps(y_grid_test))
print('CRPS of EasyUQ: %f' %crps_idr)
ll_train, h, df = optimize_paras_onefit(preds_train, y_grid_train)
ll = llscore_idr(preds_test, y_grid_test, h, df)
print('Fitted df = %d' %df , 'and h = %f' %h)
print('log score: %f' %ll)

CRPS of EasyUQ: 0.362215


### 3. Conformal predictions

Smoothing applied by using a fixed gaussian kernel and silverman's rule of thumb which is applied to each CP ensemble individually

In [2]:
from helper_functions import rot, algo72_ensemble, llscore_cp

In [6]:
# load via xarray:
cnn_forecast_train = xr.open_dataset('./data/cnn_fct_train.nc')
cnn_forecast_test = xr.open_dataset('./data/cnn_fct_test.nc')
y_train = xr.open_dataset('./data/cnn_obs_train.nc')
y_test = xr.open_dataset('./data/cnn_obs_test.nc')

# select one grid point:
lat = 8.4375
lon = 180
cnn_fct_grid_train = cnn_forecast_train.sel(lat = lat, lon = lon)['t'].values
cnn_fct_grid_test = cnn_forecast_test.sel(lat = lat, lon = lon)['t'].values
y_grid_train = y_train.sel(lat = lat, lon = lon)['t'].values
y_grid_test = y_test.sel(lat = lat, lon = lon)['t'].values

In [4]:
ens_cp = algo72_ensemble(cnn_fct_grid_train, cnn_fct_grid_test, y_grid_train)

In [8]:
crps_ens = np.mean(ps.crps_ensemble(y_grid_test, ens_cp))
print('CRPS of CP: %f' %crps_ens)

CRPS of CP: 0.362770


In [9]:
h_rot = rot(ens_cp)
ll = llscore_cp(ens_cp, y_grid_test, h_rot)
print('log score: %f' %ll)

log score: 0.994697
